# **Political Voting Survey Dashboard 🌏**

**Introduction**

This project aims to showcase my skills in Data Visualizations and Dashboard Development with Tableau using generated dummy data with an example case in political surveys. Also, while the Visualization using Tableau are using CSV Files, due to Tableau Public Limitations, I am also going to develop an ingestion process from Generated Data to PostgreSQL for further real-time (or updated) visualizations.

Due to current limitations in access data, I will generate dummy data using APIs from multiple random generators with a location in Indonesia. This project does not at all represent any political fact about any region and are artificial created for learning and skills showcasing purposes.

**Tables:**

- Votes (6000 Unique Values)
    - voter_name
    - voter_id
    - candidate_id
    - region_id
- Candidate (4 Unique Values)
    - Candidate_id
    - Nama (Buat 4)
    - Party_id (Buat 4)
- region (33 Unique Values)
    - region_id
    - nama_wilayah
- Electoral Vote ← Voters (dibuat dari yang menang masing-masing)
    - Wilayah (Ada 33)
    - Vote Count
    - Candidate_name
    - Party
- Party
    - party_id
    - party_name
    - member_count

In [ ]:
import pandas as pd
import random
from helper.data_generate import generate_person

**Region** 

In [ ]:
region_df = pd.read_csv('sources/regions_id.csv')
region_df = region_df.reset_index().rename(columns={"index": "id"})

region_df.rename(columns={"Provinsi di Indonesia":"province",
                          "Jumlah Penduduk Menurut Provinsi di Indonesia (Ribu Jiwa)":"population_count_in_thousands"},
                 inplace=True)

region_df = region_df[~region_df["province"].isin(["Indonesia"])]
region_df['id'] = region_df['id'] + 1

In [ ]:
region_df

**Party** 

In [ ]:
party_names = ["Partai Sosial Demokrasi Indonesia", "Partai Republik Nasionalis Nusantara", "Partai Amanat Indonesia", "Partai Kerjasama dan Utusan Rakyat"]
party_list = []
id = 1

for i in party_names:
    party = {}
    party["id"] = id
    id += 1
    party["party_name"] = i
    party["total_members"] = random.randint(1500, 5000)

    party_list.append(party)

In [ ]:
party_df = pd.DataFrame(party_list)

In [ ]:
party_df

**Candidate** 

In [ ]:
generate_person()

In [ ]:
candidates = []

for i in party_df["id"]:
    candidate = {}
    person = generate_person()

    candidate['id'] = i
    candidate['name'] = person['name']['first'] + ' ' + person['name']['last']
    candidate['gender'] = person['gender']
    candidate['party_id'] = i

    candidates.append(candidate)

candidates_df = pd.DataFrame(candidates) 

In [ ]:
candidates_df

**Voters** 

In [ ]:
voters_csv_df = pd.read_csv("csv/voters.csv")
voters_csv_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
voters_csv_df

In [ ]:
voters = []

def generate_voter():
    person = generate_person()
    voter = {}

    voter['name'] = person['name']['first'] + ' ' + person['name']['last']
    voter['gender'] = person['gender']
    voter['candidate_id'] = random.choice(candidates_df['id'].tolist())
    voter['region_id'] = random.choice(region_df['id'].tolist())

    print(voter)
    return voter


In [ ]:
for x in range(7):
    for i in range(250):
        print(str(i) + " " + str(x))
        voter = generate_voter()
        voters.append(voter)

In [ ]:
voters_df = pd.DataFrame(voters)

In [ ]:
voters_csv_df = pd.concat([voters_csv_df, voters_df], ignore_index=True)
voters_csv_df = voters_csv_df.drop_duplicates()
voters_csv_df.rename(columns={
    'vote': 'candidate_id'
}, inplace=True)

voters_csv_df.to_csv("csv/voters.csv")

In [ ]:
# import numpy as np

# voters_csv_df['vote'] = np.random.randint(1, max(candidates_df['id']) + 1, size=len(voters_csv_df))
# voters_csv_df['region_id'] = np.random.randint(1, max(region_df['id']) + 1, size=len(voters_csv_df))

In [ ]:
voters_csv_df

**Electoral** 

In [ ]:
votes_with_candidates = pd.merge(voters_csv_df, candidates_df[['id', 'name', 'party_id']], left_on='candidate_id', right_on='id', how='inner')

votes_with_region = pd.merge(votes_with_candidates, region_df[['id', 'province']], left_on='region_id', right_on='id', how='inner')

votes_cleansed = pd.merge(votes_with_region, party_df[['id','party_name']], left_on='party_id', right_on='id', how='inner')

votes_cleansed.drop(columns=['id_x','id_y','id'], inplace=True)

votes_cleansed.rename(columns={'name_x': 'voter_name', 'name_y':'candidate_name', 'vote':'candidate_id'}, inplace=True)

votes_cleansed

In [ ]:
# Result total pemenang
votes_cleansed.groupby('candidate_name').agg({
    'voter_name':'count'
})

In [ ]:
votes_cleansed.to_csv('csv/votes_cleansed.csv')

In [ ]:
# Data detail untuk masing-masing provinsi
details_province = votes_cleansed.groupby(['province', 'candidate_name']).agg({
    'voter_name': ['count']
})

details_province.rename(columns={'voter_name': 'vote_count'}, inplace=True)
details_province.columns = details_province.columns.droplevel(1)
details_province.reset_index()

details_province

In [ ]:
# Data pememenang setiap provinsi
winners = details_province.groupby('province')['vote_count'].idxmax()
winning_candidates = details_province.loc[winners].reset_index()
winning_candidates.to_csv('csv/regional_results.csv')

winning_candidates

In [ ]:
# Result total pemenang
winning_candidates.groupby('candidate_name').agg({
    'province':'count'
})